<a href="https://colab.research.google.com/github/anthonyibr24/Feature-Selection-Methods/blob/Htru2/htru2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np

from ucimlrepo import fetch_ucirepo

# fetch dataset
htru2 = fetch_ucirepo(id=372)

# data (as pandas dataframes)
X = htru2.data.features
y = htru2.data.targets

# metadata
print(htru2.metadata)

# variable information
print(htru2.variables)





df=pd.concat([X,y],axis=1)




{'uci_id': 372, 'name': 'HTRU2', 'repository_url': 'https://archive.ics.uci.edu/dataset/372/htru2', 'data_url': 'https://archive.ics.uci.edu/static/public/372/data.csv', 'abstract': 'Pulsar candidates collected during the HTRU survey. Pulsars are a type of star, of considerable scientific interest. Candidates must be classified in to pulsar and non-pulsar classes to aid discovery.', 'area': 'Physics and Chemistry', 'tasks': ['Classification', 'Clustering'], 'characteristics': ['Multivariate'], 'num_instances': 17898, 'num_features': 8, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2015, 'last_updated': 'Wed Apr 03 2024', 'dataset_doi': '10.24432/C5DK6R', 'creators': ['Robert Lyon'], 'intro_paper': {'ID': 460, 'type': 'NATIVE', 'title': 'Fifty years of pulsar candidate selection: from simple filters to a new principled real-time classification approach', 'a

In [ ]:
df.head(200)

,Profile_mean,Profile_stdev,Profile_skewness,Profile_kurtosis,DM_mean,DM_stdev,DM_skewness,DM_kurtosis,class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0
...,...,...,...,...,...,...,...,...,...
195,126.101562,52.567344,0.216220,-0.347581,23.096154,53.595116,2.296400,4.183569,0
196,133.046875,51.122381,-0.145586,-0.325246,1.190635,13.384291,13.871965,214.798449,0
197,123.820312,55.164163,0.016799,-0.928209,4.076087,22.256487,6.717676,50.203462,0
198,80.335938,37.698491,0.713377,2.334908,1.577759,12.400095,11.761941,180.001201,0


In [ ]:
print(df['class'].value_counts())

class
0    16259
1     1639
Name: count, dtype: int64


In [ ]:
df.shape

(17898, 9)

In [ ]:
from sklearn.utils import Bunch

# Create a Bunch-like object for compatibility
data_set = Bunch(
    data=df.drop('class', axis=1).values,
    target=df['class'].values,
    feature_names=df.drop('class', axis=1).columns.tolist(),
)


#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

#scale mean=0 std=1
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
df=df.drop('target',axis=1)
df[data.feature_names]=scaler.fit_transform(df[data.feature_names])

df['target']=data.target
df.describe().round(3)

,Profile_mean,Profile_stdev,Profile_skewness,Profile_kurtosis,DM_mean,DM_stdev,DM_skewness,DM_kurtosis,target
count,17898.000,17898.000,17898.000,17898.000,17898.000,17898.000,17898.000,17898.000,17898.000
mean,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.092
std,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.288
min,-4.104,-3.182,-2.212,-0.578,-0.421,-0.974,-2.539,-1.003,0.000
25%,-0.396,-0.610,-0.424,-0.318,-0.363,-0.611,-0.560,-0.656,0.000
50%,0.156,0.058,-0.239,-0.255,-0.333,-0.404,0.029,-0.205,0.000
75%,0.624,0.654,-0.004,-0.137,-0.243,0.108,0.532,0.323,0.000
max,3.179,7.633,7.135,10.755,7.152,4.331,5.823,10.197,1.000


# Full features

In [ ]:
#Cross validation
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression

lr=LogisticRegression(max_iter=2000)

cv_results = cross_validate(
    lr,
    df.drop('target', axis=1),
    df['target'],
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
    return_train_score=False,

)

print("\nAverage Metrics Across 4 Folds:")
print(f"Accuracy:  {np.mean(cv_results['test_accuracy']):.4f} ± {np.std(cv_results['test_accuracy']):.4f}")
print(f"Precision: {np.mean(cv_results['test_precision_macro']):.4f} ± {np.std(cv_results['test_precision_macro']):.4f}")
print(f"Recall:    {np.mean(cv_results['test_recall_macro']):.4f} ± {np.std(cv_results['test_recall_macro']):.4f}")
print(f"F1 Score:  {np.mean(cv_results['test_f1_macro']):.4f} ± {np.std(cv_results['test_f1_macro']):.4f}")


Average Metrics Across 4 Folds:
Accuracy:  0.9782 ± 0.0028
Precision: 0.9610 ± 0.0040
Recall:    0.9044 ± 0.0188
F1 Score:  0.9300 ± 0.0106


# Mutual Gain

In [ ]:
#Mutual gain
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']



In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectKBest(mutual_info_classif, k=4)),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.8s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.8s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.5s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.5s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.9768 ± 0.0026
Precision_macro: 0.9586 ± 0.0046
Recall_macro: 0.8975 ± 0.0189
F1_macro: 0.9250 ± 0.0104


In [ ]:
#ANOVA
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']

#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectKBest(score_func=f_classif, k=4)),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.2s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.9768 ± 0.0026
Precision_macro: 0.9574 ± 0.0066
Recall_macro: 0.8989 ± 0.0201
F1_macro: 0.9252 ± 0.0106


# Wrapper Methods

# Forward

In [ ]:
#FORWARD
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier

#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']

knn=KNeighborsClassifier(5)



In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SequentialFeatureSelector(knn, n_features_to_select=4, direction='forward', scoring='accuracy')),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")



[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  20.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  19.5s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  19.7s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  19.4s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.9772 ± 0.0031
Precision_macro: 0.9614 ± 0.0053
Recall_macro: 0.8978 ± 0.0212
F1_macro: 0.9262 ± 0.0118


# BACKWARD

In [ ]:
#BACKWARD
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier

#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']

knn=KNeighborsClassifier(5)

In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SequentialFeatureSelector(knn, n_features_to_select=4, direction='backward', scoring='accuracy')),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")


[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  24.3s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  23.9s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  23.5s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=  22.8s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.9775 ± 0.0029
Precision_macro: 0.9597 ± 0.0023
Recall_macro: 0.9009 ± 0.0186
F1_macro: 0.9275 ± 0.0110


# RFE

In [ ]:
#RFE
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']



In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', RFE(estimator=SVC(kernel='linear'), n_features_to_select=4)),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")


[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   3.2s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   2.4s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   3.2s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   2.5s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.9763 ± 0.0025
Precision_macro: 0.9581 ± 0.0070
Recall_macro: 0.8951 ± 0.0203
F1_macro: 0.9232 ± 0.0104


# Embeded Methods

# Random Forest

In [ ]:
#RF
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel


#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']


In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectFromModel(RandomForestClassifier(random_state=42), max_features=4)),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   5.7s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   3.5s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   3.7s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.2s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   3.9s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.9764 ± 0.0027
Precision_macro: 0.9606 ± 0.0059
Recall_macro: 0.8935 ± 0.0184
F1_macro: 0.9234 ± 0.0106


# Logistic using L1 penalty

In [ ]:
#LR L1 penalty
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest


#load dataset
data=data_set
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']


In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectFromModel(LogisticRegression(penalty='l1',solver='saga',C=0.1,max_iter=3000,random_state=42),max_features=4)),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.7s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.2s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   1.2s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.9s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s

Average Scores Across 4 Folds:
Accuracy: 0.9777 ± 0.0029
Precision_macro: 0.9600 ± 0.0029
Recall_macro: 0.9022 ± 0.0193
F1_macro: 0.9283 ± 0.0112
